In [1]:
import os
os.getcwd()

'/home/yes24/capstone/추천알고리즘/preprocessed'

In [2]:
os.chdir('/home/yes24/models/research/deep_contextual_bandits')

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from absl import app
from absl import flags

In [4]:
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling

In [5]:
from bandits.data.data_sampler import sample_census_data
from bandits.data.data_sampler import one_hot,safe_std,remove_underrepresented_classes,classification_to_bandit_problem

In [6]:
def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
    """Displays summary statistics of the performance of each algorithm."""

    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('{} bandit completed after {} seconds.'.format(
        name, time.time() - t_init))
    print('---------------------------------------------------')

    performance_pairs = []
    for j, a in enumerate(algos):
        performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
    performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
    for i, (name, reward) in enumerate(performance_pairs):
        print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

    print('---------------------------------------------------')
    print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
    print('Frequency of optimal actions (action, frequency):')
    print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
    print('---------------------------------------------------')
    print('---------------------------------------------------')



In [7]:
num_actions = 5

In [8]:
df_100_fin = pd.read_csv('df100_test_0701.csv')

In [9]:
df_100_fin['purchase'].sum()

706.0

In [10]:
df_context = df_100_fin.iloc[:,6:]
labels = df_100_fin.iloc[:,4].astype('int8').values

In [11]:
df_context

,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,most_pref_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [12]:
df_context, labels = remove_underrepresented_classes(df_context, labels)
contexts = df_context.values

Unique classes and their ratio of total: {0: 0.0931526736027099, 1: 0.18872489716912655, 2: 0.17735301233970482, 3: 0.432615533510767, 4: 0.10815388337769175}


In [13]:
sampled_vals = classification_to_bandit_problem(contexts, labels, num_actions)

In [14]:
contexts, rewards, (opt_rewards, opt_actions) = sampled_vals

In [15]:
rewards_list = df_100_fin['purchase'].values.tolist()

In [16]:
rewards2 = rewards.copy()

In [17]:
for i in range(len(rewards_list)):
    if rewards_list[i] == 0:
        rewards2[i] = np.where(rewards2[i]==1,0,rewards2[i])

In [18]:
dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [19]:
dataset2 = np.hstack((contexts, rewards2))
context_dim = contexts.shape[1]

In [20]:
mu = np.load('/home/yes24/capstone/소라공유폴더/mean_0701.npy')
cov = np.load('/home/yes24/capstone/소라공유폴더/cov_0701.npy')

In [29]:
hparams = tf.contrib.training.HParams(num_actions=num_actions)
hparams_linear = tf.contrib.training.HParams(num_actions=num_actions,
                                               context_dim=context_dim,
                                               a0=6,
                                               b0=6,
                                               lambda_prior=0.25,
                                               initial_pulls=1,
                                            mu=mu,
                                            cov=cov)

In [30]:
algos = [LinearFullPosteriorSampling('LinFullPost', hparams_linear)]

In [31]:
run_contextual_bandit(context_dim, num_actions, dataset2, algos)

(array([[0.],
        [1.],
        [2.],
        ...,
        [4.],
        [4.],
        [4.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 [])

In [23]:
t_init2 = time.time()
results2 = run_contextual_bandit(context_dim, num_actions, dataset2, algos)


In [30]:
h_rewards2.T[0].sum()

82.0

In [27]:
_2, h_rewards2,h_vals2 = results2

In [24]:
h_vals2

[]

In [25]:
t_init = time.time()
results = run_contextual_bandit(context_dim, num_actions, dataset, algos)
_, h_rewards = results

In [28]:
display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 42.18470048904419 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =     3548.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [29]:
display_results(algos, opt_rewards, opt_actions, h_rewards2, t_init2, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 22.114109992980957 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =      672.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [31]:
display_results(algos, opt_rewards, opt_actions, h_rewards2, t_init2, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 93.9321277141571 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =       82.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [31]:
ts_actions = _.T[0]

In [36]:
len(ts_actions)

4133

In [37]:
len(labels)

4133

In [40]:
len(rewards_list)

4133

In [45]:
rewards_list = np.array(rewards_list)

In [50]:
df_eval = pd.concat([pd.DataFrame(ts_actions),pd.DataFrame(labels),pd.DataFrame(rewards_list)],axis=1)

In [52]:
df_eval.columns=['ts_actions','labels','reward']

In [60]:
rewards_fin = 0
for i in range(len(df_eval)):
    if df_eval['reward'][i] == 1:
        if df_eval['ts_actions'][i] == df_eval['labels'][i]:
            rewards_fin +=1

In [61]:
rewards_fin

615

In [68]:
mean1 = algos[0].mu

In [69]:
cov1 = algos[0].cov

In [75]:
mean2 = algos[0].mu

In [76]:
cov2 = algos[0].cov

In [31]:
algos[0].mu

[array([ 2.94238351e-03, -2.94238351e-03,  0.00000000e+00,  1.86575911e-02,
        -2.70227944e-02, -1.15259194e-02,  2.31228639e-02,  5.91907131e-03,
         0.00000000e+00, -2.09351004e-02,  5.09044305e-03, -2.21060517e-02,
        -1.42324452e-02,  5.62984922e-05, -2.10951052e-02,  2.05857716e-03,
         5.08096442e-03, -8.19246790e-03,  1.84492896e-02,  0.00000000e+00,
         1.05237238e-02,  1.97907673e-02,  0.00000000e+00, -2.15151753e-02,
        -1.46805238e-02, -1.08972048e-02, -4.87409864e-03, -1.37862495e-02,
         2.45011192e-02, -6.91920123e-03,  2.83958830e-02, -2.15381536e-03,
         9.58653860e-03, -4.11546726e-03, -1.32562954e-03,  1.32562954e-03,
         1.69998275e-02, -1.69998275e-02, -1.48153127e-03,  1.00725578e-02,
         8.28318908e-02, -1.44173775e-02,  3.27791961e-02,  3.81069300e-02,
         1.08810215e-02,  1.08113050e-02, -9.18157155e-03,  3.27987480e-02,
         1.06950729e-02,  2.76052955e-02,  5.91797466e-02,  6.10902328e-03,
         3.1

In [8]:
df_100 = pd.read_csv('/home/yes24/capstone/영송공유폴더/test_100.csv')
df_100

,account_id,product_id,price,purchase,book_cluster,user_cluster
0,113512,89940862,0.0,0.0,4.0,3.0
1,113512,84434768,0.0,0.0,2.0,3.0
2,113512,74393338,0.0,0.0,2.0,3.0
3,113512,90108425,0.0,0.0,3.0,3.0
4,113512,85927898,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0
4129,16790454,89222585,14400.0,1.0,3.0,1.0
4130,16790454,89903437,0.0,0.0,3.0,1.0
4131,16790454,90114596,0.0,0.0,3.0,1.0


In [13]:
user_fin = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/user_dummified_1df_0628.csv')

In [14]:
book_fin = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/bookfeature_dummified_0628.csv')
book_fin.drop(columns=book_fin.columns[0],inplace=True)

In [15]:
user100_list = df_100['account_id'].unique()

In [16]:
pro100_list = df_100['product_id'].unique()

In [17]:
user_fin[user_fin['account_id'].isin(user100_list)]

,account_id,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
12301,231022.0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
13217,2232676.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,1
13641,3300415.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
28128,9805328.0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
49539,2122933.0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166997,1569435.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
1167226,2332587.0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,1
1204153,3056585.0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
1209537,1176113.0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,1


In [18]:
book_fin[book_fin['product_id'].isin(pro100_list)]

,product_id,cat1.0,cat2.0,cat3.0,cat4.0,cat5.0,cat7.0,cat8.0,cat9.0,cat10.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
236,55868083,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
246,55869065,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
340,55936772,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
348,55944631,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
374,55976113,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269937,85030922,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270155,85108883,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270172,85109178,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270304,85112006,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [19]:
user_context_add = pd.merge(df_100,user_fin,on='account_id',how='left')
user_context_add

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0


In [20]:
df_100_fin = pd.merge(user_context_add, book_fin, on='product_id',how='left')
df_100_fin

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [63]:
df_100_fin.to_csv('df100_test_0701.csv',index=False)